In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [23]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_CB")

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [25]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [27]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [28]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [29]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [31]:
from transformers import set_seed

set_seed(42)

In [32]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_cb_bert"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [34]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.7239, 'grad_norm': 4.290432929992676, 'learning_rate': 1.7500000000000002e-05, 'epoch': 3.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.2944774627685547, 'eval_accuracy': 0.3181818181818182, 'eval_f1': 0.1536050156739812, 'eval_runtime': 1.1041, 'eval_samples_per_second': 19.927, 'eval_steps_per_second': 2.717, 'epoch': 3.57}
{'loss': 0.3879, 'grad_norm': 2.0067036151885986, 'learning_rate': 1.5000000000000002e-05, 'epoch': 7.14}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 4.316608905792236, 'eval_accuracy': 0.36363636363636365, 'eval_f1': 0.24300699300699302, 'eval_runtime': 0.2913, 'eval_samples_per_second': 75.511, 'eval_steps_per_second': 10.297, 'epoch': 25.0}
{'loss': 0.0032, 'grad_norm': 0.07383772730827332, 'learning_rate': 0.0, 'epoch': 28.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 4.216930866241455, 'eval_accuracy': 0.36363636363636365, 'eval_f1': 0.24300699300699302, 'eval_runtime': 0.2989, 'eval_samples_per_second': 73.615, 'eval_steps_per_second': 10.038, 'epoch': 28.57}
{'train_runtime': 167.0859, 'train_samples_per_second': 19.152, 'train_steps_per_second': 2.394, 'train_loss': 0.17338078800588846, 'epoch': 28.57}


TrainOutput(global_step=400, training_loss=0.17338078800588846, metrics={'train_runtime': 167.0859, 'train_samples_per_second': 19.152, 'train_steps_per_second': 2.394, 'total_flos': 303236158357944.0, 'train_loss': 0.17338078800588846, 'epoch': 28.571428571428573})

In [35]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 4.216930866241455,
 'eval_accuracy': 0.36363636363636365,
 'eval_f1': 0.24300699300699302,
 'eval_runtime': 0.2927,
 'eval_samples_per_second': 75.167,
 'eval_steps_per_second': 10.25,
 'epoch': 28.571428571428573}

In [36]:
trainer.push_to_hub(new_model_name)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_cb_bert/commit/79ac49943400b6b1e18060730e55aefd38f8c413', commit_message='fine_tuned_cb_bert', commit_description='', oid='79ac49943400b6b1e18060730e55aefd38f8c413', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [38]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)
import torch

adapter_name = "lenatr99/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [39]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
outputs = model(**inputs, labels=label)
logits = outputs.logits

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# Print prediction
logits.argmax().item()

1